# 🔮 건설용 자갈 암석 분류 AI - 추론 및 제출 파일 생성

## 📋 목표
- 학습된 모델을 사용하여 테스트 데이터 예측
- 대회 제출 파일 생성 (sample_submission.csv 형식)
- 예측 결과 분석 및 품질 검증

## 🎯 진행 과정
1. 학습된 모델 로드
2. 테스트 데이터 전처리 파이프라인
3. 배치 단위 추론 실행
4. 제출 파일 생성
5. 예측 결과 분석


In [1]:
# ============================================================================
# 📦 필수 라이브러리 및 설정
# ============================================================================

import os
import sys
import json
import warnings
from pathlib import Path
from datetime import datetime

# 데이터 처리
import pandas as pd
import numpy as np
from PIL import Image

# 딥러닝
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import timm

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager

# 경고 억제
warnings.filterwarnings('ignore')

# 한글 폰트 설정
def setup_korean_font():
    """Windows 환경에서 한글 폰트 설정"""
    font_names = ['Malgun Gothic', 'NanumGothic', 'DejaVu Sans']
    
    for font_name in font_names:
        try:
            font_path = None
            for font in font_manager.fontManager.ttflist:
                if font_name.lower() in font.name.lower():
                    font_path = font.fname
                    break
            
            if font_path:
                plt.rcParams['font.family'] = font_name
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 한글 폰트 설정 완료: {font_name}")
                return True
        except:
            continue
    
    print("⚠️ 한글 폰트 설정 실패 - 기본 폰트 사용")
    return False

setup_korean_font()

# 시드 고정
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print("🔮 건설용 자갈 암석 분류 AI - 추론 시작!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("=" * 60)


c:\Users\wang\anaconda3\envs\yolo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 한글 폰트 설정 완료: Malgun Gothic
🔮 건설용 자갈 암석 분류 AI - 추론 시작!
PyTorch 버전: 2.2.2+cu121
CUDA 사용 가능: True
GPU: NVIDIA GeForce RTX 4070


In [2]:
# ============================================================================
# 🔧 경로 및 설정 로드
# ============================================================================

# 기본 경로 설정
BASE_PATH = r"D:\data\stones\open"
TEST_PATH = os.path.join(BASE_PATH, "test")
TEST_CSV_PATH = os.path.join(BASE_PATH, "test.csv")
SUBMISSION_PATH = os.path.join(BASE_PATH, "sample_submission.csv")

# 프로젝트 경로
MODEL_DIR = Path("../models")
EXPERIMENT_DIR = Path("../experiments")
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(exist_ok=True)

# 실험 설정 로드
config_path = EXPERIMENT_DIR / "experiment_config.json"
with open(config_path, 'r', encoding='utf-8') as f:
    config = json.load(f)

# 클래스 정보
CLASS_NAMES = config['project_info']['class_names']
NUM_CLASSES = config['project_info']['num_classes']
class_to_idx = {cls: i for i, cls in enumerate(CLASS_NAMES)}
idx_to_class = {i: cls for cls, i in class_to_idx.items()}

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("📂 경로 설정 완료:")
print(f"테스트 데이터: {TEST_PATH}")
print(f"테스트 CSV: {TEST_CSV_PATH}")
print(f"모델 저장: {MODEL_DIR}")
print(f"결과 저장: {RESULTS_DIR}")

print(f"\n🏷️ 클래스 정보:")
print(f"클래스 수: {NUM_CLASSES}")
print(f"클래스 목록: {CLASS_NAMES}")

print(f"\n💻 연산 장치: {device}")
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU 메모리: {gpu_memory:.1f} GB")
print("=" * 60)


📂 경로 설정 완료:
테스트 데이터: D:\data\stones\open\test
테스트 CSV: D:\data\stones\open\test.csv
모델 저장: ..\models
결과 저장: ..\results

🏷️ 클래스 정보:
클래스 수: 7
클래스 목록: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']

💻 연산 장치: cuda
GPU 메모리: 12.0 GB


In [3]:
# ============================================================================
# 🏭 모델 팩토리 및 로더
# ============================================================================

class ModelFactory:
    """다양한 모델 아키텍처를 생성하는 팩토리 클래스"""
    
    @staticmethod
    def create_model(model_name, num_classes=7, pretrained=False):
        """모델 생성"""
        try:
            # timm을 통한 모델 생성
            model = timm.create_model(
                model_name,
                pretrained=pretrained,
                num_classes=num_classes
            )
            return model
        except Exception as e:
            print(f"❌ 모델 생성 실패 ({model_name}): {e}")
            return None

class ModelLoader:
    """학습된 모델을 로드하는 클래스"""
    
    def __init__(self, device):
        self.device = device
        self.model = None
        self.model_info = None
    
    def load_model(self, model_path, model_name):
        """저장된 모델 로드"""
        try:
            print(f"📂 모델 로드 중: {model_path}")
            
            # 체크포인트 로드
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # 모델 생성
            self.model = ModelFactory.create_model(model_name, NUM_CLASSES)
            if self.model is None:
                return False
            
            # 가중치 로드
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.model.to(self.device)
            self.model.eval()
            
            # 모델 정보 저장
            self.model_info = {
                'model_path': str(model_path),
                'model_name': model_name,
                'best_score': checkpoint.get('best_score', 'N/A'),
                'epoch': checkpoint.get('epoch', 'N/A'),
                'total_params': sum(p.numel() for p in self.model.parameters())
            }
            
            print(f"✅ 모델 로드 완료!")
            print(f"   모델 아키텍처: {model_name}")
            print(f"   최고 성능: {self.model_info['best_score']}")
            print(f"   총 파라미터: {self.model_info['total_params']:,}개")
            
            return True
            
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return False
    
    def get_available_models(self):
        """사용 가능한 모델 목록 반환"""
        model_files = list(MODEL_DIR.glob("*.pth"))
        
        available = []
        for model_file in model_files:
            # 파일명에서 모델 정보 추출
            model_info = {
                'path': model_file,
                'name': model_file.stem,
                'size_mb': model_file.stat().st_size / (1024 * 1024),
                'modified': datetime.fromtimestamp(model_file.stat().st_mtime)
            }
            available.append(model_info)
        
        # 수정 시간순 정렬 (최신 순)
        available.sort(key=lambda x: x['modified'], reverse=True)
        
        return available

# 사용 가능한 모델 확인
model_loader = ModelLoader(device)
available_models = model_loader.get_available_models()

print("🔍 사용 가능한 모델:")
if available_models:
    for i, model_info in enumerate(available_models, 1):
        print(f"{i}. {model_info['name']}")
        print(f"   크기: {model_info['size_mb']:.1f} MB")
        print(f"   수정: {model_info['modified'].strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("❌ 사용 가능한 모델이 없습니다!")
    print("   먼저 03_model_training.ipynb에서 모델을 학습하세요.")


🔍 사용 가능한 모델:
1. EfficientNetV2-S_quick_baseline
   크기: 232.3 MB
   수정: 2025-08-23 04:55:16


In [4]:
# ============================================================================
# 🔮 학습된 모델 로드
# ============================================================================

if not available_models:
    print("❌ 로드할 모델이 없습니다!")
    print("   먼저 03_model_training.ipynb에서 모델을 학습하세요.")
else:
    # 가장 최신 모델 선택
    latest_model = available_models[0]
    model_path = latest_model['path']
    
    print(f"📂 선택된 모델: {latest_model['name']}")
    print(f"   경로: {model_path}")
    print(f"   크기: {latest_model['size_mb']:.1f} MB")
    
    # 모델명 추출 (파일명에서)
    if "EfficientNetV2-S" in latest_model['name']:
        model_name = "tf_efficientnetv2_s.in21k_ft_in1k"
    elif "EfficientNetV2-M" in latest_model['name']:
        model_name = "tf_efficientnetv2_m.in21k_ft_in1k"
    elif "ConvNeXt" in latest_model['name']:
        model_name = "convnext_tiny.fb_in22k_ft_in1k"
    elif "ViT" in latest_model['name']:
        model_name = "vit_tiny_patch16_224.augreg_in21k_ft_in1k"
    else:
        model_name = "tf_efficientnetv2_s.in21k_ft_in1k"  # 기본값
    
    print(f"   아키텍처: {model_name}")
    
    # 모델 로드
    success = model_loader.load_model(model_path, model_name)
    
    if success:
        print(f"\n🎯 모델 준비 완료!")
        print(f"   추론 준비 상태: ✅")
        print(f"   GPU 사용: {next(model_loader.model.parameters()).is_cuda}")
    else:
        print(f"\n❌ 모델 로드 실패!")
        model_loader.model = None


📂 선택된 모델: EfficientNetV2-S_quick_baseline
   경로: ..\models\EfficientNetV2-S_quick_baseline.pth
   크기: 232.3 MB
   아키텍처: tf_efficientnetv2_s.in21k_ft_in1k
📂 모델 로드 중: ..\models\EfficientNetV2-S_quick_baseline.pth
✅ 모델 로드 완료!
   모델 아키텍처: tf_efficientnetv2_s.in21k_ft_in1k
   최고 성능: 0.7997457306490847
   총 파라미터: 20,186,455개

🎯 모델 준비 완료!
   추론 준비 상태: ✅
   GPU 사용: True


In [5]:
# ============================================================================
# 📊 테스트 데이터셋 및 추론 파이프라인
# ============================================================================

class TestDataset(Dataset):
    """테스트 데이터를 위한 데이터셋 클래스"""
    
    def __init__(self, test_csv_path, test_dir, transform=None):
        self.test_df = pd.read_csv(test_csv_path)
        self.test_dir = test_dir
        self.transform = transform
        
        print(f"📊 테스트 데이터셋 생성 완료:")
        print(f"   총 샘플 수: {len(self.test_df):,}개")
        print(f"   CSV 컬럼: {list(self.test_df.columns)}")
        
        # 샘플 경로 확인
        sample_img_path = self.test_df.iloc[0]['img_path']
        if sample_img_path.startswith('./'):
            # ./를 제거하고 BASE_PATH와 조인
            relative_path = sample_img_path[2:]
            base_path = os.path.dirname(self.test_dir)
            sample_path = os.path.join(base_path, relative_path)
        else:
            sample_path = os.path.join(self.test_dir, sample_img_path)
            
        if os.path.exists(sample_path):
            print(f"✅ 이미지 경로 확인 완료")
        else:
            print(f"❌ 이미지 경로 오류: {sample_path}")
            print(f"   원본 img_path: {sample_img_path}")
            print(f"   test_dir: {self.test_dir}")
            print(f"   base_path: {os.path.dirname(self.test_dir)}")
    
    def __len__(self):
        return len(self.test_df)
    
    def __getitem__(self, idx):
        # 이미지 정보 가져오기
        img_id = self.test_df.iloc[idx]['ID']
        img_path = self.test_df.iloc[idx]['img_path']
        
        # 이미지 로드 (img_path가 이미 ./test/를 포함하므로 BASE_PATH 사용)
        # img_path 형태: "./test/TEST_00000.jpg"
        if img_path.startswith('./'):
            # ./를 제거하고 BASE_PATH와 조인
            relative_path = img_path[2:]  # "./test/TEST_00000.jpg" -> "test/TEST_00000.jpg"
            base_path = os.path.dirname(self.test_dir)  # D:\data\stones\open\test -> D:\data\stones\open
            full_path = os.path.join(base_path, relative_path)
        else:
            # 기존 방식 (혹시 경로 형태가 다를 경우를 위한 fallback)
            full_path = os.path.join(self.test_dir, img_path)
        
        try:
            image = Image.open(full_path).convert('RGB')
            
            # 변환 적용
            if self.transform:
                image = self.transform(image)
            
            return {
                'image': image,
                'id': img_id,
                'path': img_path
            }
            
        except Exception as e:
            print(f"❌ 이미지 로드 실패 ({img_path}): {e}")
            print(f"   시도한 경로: {full_path}")
            # 빈 이미지 반환
            dummy_image = torch.zeros(3, 224, 224)
            return {
                'image': dummy_image,
                'id': img_id,
                'path': img_path
            }

# 테스트 변환 파이프라인 (학습 시와 동일한 전처리, 증강 없음)
test_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 표준
        std=[0.229, 0.224, 0.225]
    )
])

print("🎨 테스트 변환 파이프라인:")
print("   1. Resize: 224×224 (BICUBIC)")
print("   2. ToTensor: [0,255] → [0,1]")
print("   3. Normalize: ImageNet 표준")
print("   ⚠️ 증강 없음 (일관된 예측을 위해)")


🎨 테스트 변환 파이프라인:
   1. Resize: 224×224 (BICUBIC)
   2. ToTensor: [0,255] → [0,1]
   3. Normalize: ImageNet 표준
   ⚠️ 증강 없음 (일관된 예측을 위해)


In [ ]:
# ============================================================================
# 🚀 추론 실행 및 제출 파일 생성
# ============================================================================

def run_inference_and_create_submission():
    """추론 실행 및 제출 파일 생성"""
    
    # 테스트 데이터 존재 확인
    print("🔍 테스트 데이터 확인:")
    if not os.path.exists(TEST_CSV_PATH):
        print(f"❌ test.csv 파일이 없습니다: {TEST_CSV_PATH}")
        return None
    if not os.path.exists(TEST_PATH):
        print(f"❌ 테스트 이미지 디렉토리가 없습니다: {TEST_PATH}")
        return None
    
    print(f"✅ 테스트 데이터 확인 완료")
    
    # 테스트 데이터셋 생성
    try:
        test_dataset = TestDataset(
            test_csv_path=TEST_CSV_PATH,
            test_dir=TEST_PATH,
            transform=test_transform
        )
        
        # 테스트 데이터로더 생성
        test_dataloader = DataLoader(
            test_dataset,
            batch_size=32,  # 추론용이므로 큰 배치 사이즈 사용
            shuffle=False,  # 순서 유지 필요
            num_workers=0,  # Windows 호환성
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        print(f"\n📊 테스트 DataLoader 설정:")
        print(f"   배치 크기: {test_dataloader.batch_size}")
        print(f"   전체 배치 수: {len(test_dataloader):,}개")
        print(f"   전체 샘플 수: {len(test_dataset):,}개")
        
    except Exception as e:
        print(f"❌ 테스트 데이터 로드 실패: {e}")
        return None
    
    # 모델 확인
    if model_loader.model is None:
        print("❌ 로드된 모델이 없습니다!")
        return None
    
    # 추론 실행
    print(f"\n🔮 추론 실행 중...")
    model_loader.model.eval()
    
    predictions = []
    ids = []
    confidences = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_dataloader):
            # 데이터 GPU로 이동
            images = batch['image'].to(device)
            batch_ids = batch['id']
            
            # 모델 예측
            outputs = model_loader.model(images)
            probabilities = F.softmax(outputs, dim=1)
            
            # 예측 클래스 및 신뢰도
            max_probs, predicted_classes = torch.max(probabilities, dim=1)
            
            # 결과 저장
            predictions.extend(predicted_classes.cpu().numpy())
            confidences.extend(max_probs.cpu().numpy())
            ids.extend(batch_ids)
            
            # 진행률 출력
            if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(test_dataloader):
                progress = (batch_idx + 1) / len(test_dataloader) * 100
                print(f"   진행률: {progress:.1f}% ({batch_idx + 1:,}/{len(test_dataloader):,})")
    
    print(f"✅ 추론 완료!")
    print(f"   총 예측 수: {len(predictions):,}개")
    print(f"   평균 신뢰도: {np.mean(confidences):.3f}")
    
    # 제출 파일 생성
    print(f"\n📄 제출 파일 생성 중...")
    
    # 예측 결과를 클래스명으로 변환
    predicted_classes = [idx_to_class[pred] for pred in predictions]
    
    # 제출 파일 생성
    submission_df = pd.DataFrame({
        'ID': ids,
        'rock_type': predicted_classes
    })
    
    # 정렬 (ID 순서대로)
    submission_df = submission_df.sort_values('ID').reset_index(drop=True)
    
    # 파일명에 타임스탬프 추가
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name_short = model_loader.model_info['model_name'].split('.')[0]
    submission_filename = f"submission_{model_name_short}_{timestamp}.csv"
    submission_path = RESULTS_DIR / submission_filename
    
    # 파일 저장
    submission_df.to_csv(submission_path, index=False)
    
    print(f"✅ 제출 파일 생성 완료: {submission_path}")
    print(f"   총 샘플 수: {len(submission_df):,}개")
    print(f"   파일 크기: {os.path.getsize(submission_path) / 1024:.1f} KB")
    
    # 예측 분포 확인
    print(f"\n🏷️ 예측 클래스 분포:")
    class_counts = submission_df['rock_type'].value_counts()
    for rock_type, count in class_counts.items():
        percentage = count / len(submission_df) * 100
        print(f"   {rock_type}: {count:,}개 ({percentage:.1f}%)")
    
    return {
        'submission_df': submission_df,
        'submission_path': submission_path,
        'predictions': predictions,
        'confidences': confidences,
        'timestamp': timestamp
    }

# 추론 및 제출 파일 생성 실행
if model_loader.model is not None:
    inference_results = run_inference_and_create_submission()
    if inference_results:
        print(f"\n🎯 추론 작업 완료!")
        print(f"   제출 파일: {inference_results['submission_path']}")
else:
    print("❌ 모델이 로드되지 않아 추론을 실행할 수 없습니다!")
    inference_results = None


🔍 테스트 데이터 확인:
✅ 테스트 데이터 확인 완료
📊 테스트 데이터셋 생성 완료:
   총 샘플 수: 95,006개
   CSV 컬럼: ['ID', 'img_path']
✅ 이미지 경로 확인 완료

📊 테스트 DataLoader 설정:
   배치 크기: 32
   전체 배치 수: 2,969개
   전체 샘플 수: 95,006개

🔮 추론 실행 중...
   진행률: 3.4% (100/2,969)
   진행률: 6.7% (200/2,969)
   진행률: 10.1% (300/2,969)
   진행률: 13.5% (400/2,969)
   진행률: 16.8% (500/2,969)
   진행률: 20.2% (600/2,969)
   진행률: 23.6% (700/2,969)
   진행률: 26.9% (800/2,969)
   진행률: 30.3% (900/2,969)
   진행률: 33.7% (1,000/2,969)
   진행률: 37.0% (1,100/2,969)
   진행률: 40.4% (1,200/2,969)
   진행률: 43.8% (1,300/2,969)


In [ ]:
# ============================================================================
# 📊 예측 결과 분석 및 시각화
# ============================================================================

def analyze_and_visualize_results(inference_results):
    """예측 결과 상세 분석 및 시각화"""
    
    if inference_results is None:
        print("❌ 분석할 추론 결과가 없습니다!")
        return
    
    submission_df = inference_results['submission_df']
    confidences = np.array(inference_results['confidences'])
    predictions = inference_results['predictions']
    timestamp = inference_results['timestamp']
    
    print("📊 예측 결과 상세 분석")
    print("=" * 60)
    
    # 신뢰도 분석
    print(f"🎯 예측 신뢰도 분석:")
    print(f"   평균 신뢰도: {np.mean(confidences):.3f}")
    print(f"   표준편차: {np.std(confidences):.3f}")
    print(f"   최고 신뢰도: {np.max(confidences):.3f}")
    print(f"   최저 신뢰도: {np.min(confidences):.3f}")
    
    # 신뢰도 구간별 분포
    confidence_ranges = [
        (0.9, 1.0, "매우 높음"),
        (0.8, 0.9, "높음"),
        (0.7, 0.8, "보통"),
        (0.6, 0.7, "낮음"),
        (0.0, 0.6, "매우 낮음")
    ]
    
    print(f"\n📈 신뢰도 구간별 분포:")
    for min_conf, max_conf, label in confidence_ranges:
        count = np.sum((confidences >= min_conf) & (confidences < max_conf))
        percentage = count / len(confidences) * 100
        print(f"   {label} ({min_conf:.1f}-{max_conf:.1f}): {count:,}개 ({percentage:.1f}%)")
    
    # 클래스별 예측 분포 시각화
    plt.figure(figsize=(16, 12))
    
    # 1. 클래스별 예측 분포
    plt.subplot(2, 3, 1)
    class_counts = submission_df['rock_type'].value_counts()
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_counts)))
    
    bars = plt.bar(range(len(class_counts)), class_counts.values, color=colors)
    plt.xticks(range(len(class_counts)), class_counts.index, rotation=45, ha='right')
    plt.title('예측 클래스별 분포', fontsize=14, fontweight='bold')
    plt.ylabel('예측 개수')
    
    # 막대 위에 개수 표시
    for bar, count in zip(bars, class_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100,
                f'{count:,}', ha='center', va='bottom', fontsize=9)
    
    # 2. 신뢰도 히스토그램
    plt.subplot(2, 3, 2)
    plt.hist(confidences, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(np.mean(confidences), color='red', linestyle='--', 
                label=f'평균: {np.mean(confidences):.3f}')
    plt.title('예측 신뢰도 분포', fontsize=14, fontweight='bold')
    plt.xlabel('신뢰도')
    plt.ylabel('빈도')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. 클래스별 평균 신뢰도
    plt.subplot(2, 3, 3)
    
    # 클래스별 평균 신뢰도 계산
    class_confidences = {}
    for i, pred_class in enumerate(predictions):
        class_name = idx_to_class[pred_class]
        if class_name not in class_confidences:
            class_confidences[class_name] = []
        class_confidences[class_name].append(confidences[i])
    
    avg_confidences = {cls: np.mean(confs) for cls, confs in class_confidences.items()}
    
    sorted_classes = sorted(avg_confidences.items(), key=lambda x: x[1], reverse=True)
    classes, avg_confs = zip(*sorted_classes)
    
    bars = plt.bar(range(len(classes)), avg_confs, color=colors[:len(classes)])
    plt.xticks(range(len(classes)), classes, rotation=45, ha='right')
    plt.title('클래스별 평균 신뢰도', fontsize=14, fontweight='bold')
    plt.ylabel('평균 신뢰도')
    plt.ylim(0, 1)
    
    # 막대 위에 값 표시
    for bar, conf in zip(bars, avg_confs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{conf:.3f}', ha='center', va='bottom', fontsize=9)
    
    # 4. 신뢰도 vs 클래스 박스플롯
    plt.subplot(2, 3, 4)
    
    # 박스플롯 데이터 준비
    box_data = [class_confidences[cls] for cls in sorted(class_confidences.keys())]
    box_labels = sorted(class_confidences.keys())
    
    plt.boxplot(box_data, labels=box_labels)
    plt.xticks(rotation=45, ha='right')
    plt.title('클래스별 신뢰도 분포', fontsize=14, fontweight='bold')
    plt.ylabel('신뢰도')
    plt.grid(True, alpha=0.3)
    
    # 5. 신뢰도 구간별 분포
    plt.subplot(2, 3, 5)
    ranges = ['매우 낮음', '낮음', '보통', '높음', '매우 높음']
    range_counts = []
    for min_conf, max_conf, _ in confidence_ranges:
        count = np.sum((confidences >= min_conf) & (confidences < max_conf))
        range_counts.append(count)
    
    plt.pie(range_counts, labels=ranges, autopct='%1.1f%%', colors=plt.cm.Pastel1.colors)
    plt.title('신뢰도 구간별 분포', fontsize=14, fontweight='bold')
    
    # 6. 클래스별 예측 분포 (파이 차트)
    plt.subplot(2, 3, 6)
    plt.pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', colors=colors)
    plt.title('예측 클래스 분포', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    
    # 결과 저장
    analysis_plot_path = RESULTS_DIR / f"prediction_analysis_{timestamp}.png"
    plt.savefig(analysis_plot_path, dpi=300, bbox_inches='tight')
    print(f"\n📊 분석 차트 저장: {analysis_plot_path}")
    
    plt.show()
    
    return class_confidences, avg_confidences

def generate_final_report(inference_results):
    """최종 추론 리포트 생성"""
    print("\n" + "="*80)
    print("🎯 건설용 자갈 암석 분류 AI - 추론 완료 리포트")
    print("="*80)
    
    # 모델 정보
    if model_loader.model_info:
        print(f"\n🤖 사용된 모델:")
        print(f"   아키텍처: {model_loader.model_info['model_name']}")
        print(f"   체크포인트: {os.path.basename(model_loader.model_info['model_path'])}")
        print(f"   최고 성능: {model_loader.model_info['best_score']}")
        print(f"   총 파라미터: {model_loader.model_info['total_params']:,}개")
    
    # 추론 결과
    if inference_results:
        submission_df = inference_results['submission_df']
        confidences = inference_results['confidences']
        
        print(f"\n📊 추론 결과:")
        print(f"   총 예측 샘플: {len(submission_df):,}개")
        print(f"   평균 신뢰도: {np.mean(confidences):.3f}")
        print(f"   신뢰도 범위: {np.min(confidences):.3f} - {np.max(confidences):.3f}")
        
        # 제출 파일 정보
        print(f"\n📄 제출 파일:")
        submission_path = inference_results['submission_path']
        print(f"   파일명: {os.path.basename(submission_path)}")
        print(f"   경로: {submission_path}")
        print(f"   샘플 수: {len(submission_df):,}개")
        print(f"   파일 크기: {os.path.getsize(submission_path) / 1024:.1f} KB")
        
        # 예측 클래스 분포
        print(f"\n🏷️ 예측 클래스 분포:")
        class_counts = submission_df['rock_type'].value_counts()
        for rock_type, count in class_counts.items():
            percentage = count / len(submission_df) * 100
            print(f"   {rock_type}: {count:,}개 ({percentage:.1f}%)")
        
        # 생성된 파일들
        print(f"\n📁 생성된 파일들:")
        result_files = list(RESULTS_DIR.glob(f"*{inference_results['timestamp']}*"))
        for file_path in result_files:
            print(f"   📄 {file_path.name}")
        
        print(f"\n✅ 추론 작업 완료!")
        print(f"   다음 단계: 제출 파일을 대회 사이트에 업로드")
        print(f"   경로: {submission_path}")
    
    print("="*80)

# 예측 결과 분석 실행
if inference_results is not None:
    print("\n🔍 예측 결과 분석 시작...")
    class_confidences, avg_confidences = analyze_and_visualize_results(inference_results)
    generate_final_report(inference_results)
else:
    print("❌ 분석할 추론 결과가 없습니다!")
    print("   위의 셀을 실행하여 추론을 완료하세요.")
